In [1]:
import requests
import json
import polars as pl
import get_ca_data as get_ca

In [2]:
ca_lsoa_codes = get_ca.get_ca_lsoa_codes(
    get_ca.get_postcode_df(
        get_ca.get_zipped_csv_file(),
         get_ca.get_ca_la_codes(
            get_ca.get_ca_la_df(year = 2023)
            )
            )
            )

In [18]:
# record limit of 2000 makes this useless
base_url = "https://services3.arcgis.com/ivmBBrHfQfDnDf8Q/arcgis/rest/services/Indices_of_Multiple_Deprivation_(IMD)_2019/FeatureServer/0/query"

In [3]:
# lets use data.gov.uk lookups

url_imd_csv = "https://open-geography-portalx-ons.hub.arcgis.com/datasets/ons::index-of-multiple-deprivation-dec-2019-lookup-in-england.csv?where=1=1&outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"

In [4]:
imd_df_raw = pl.read_csv(url_imd_csv)

In [5]:
 def remove_numbers(input_string):
        # rename columns
        # Create a translation table that maps each digit to None
        lowercase_string = input_string.lower()
        translation_table = str.maketrans("", "", "0123456789")
        # Use the translation table to remove all numbers from the input string
        result_string = lowercase_string.translate(translation_table)
        return result_string

In [6]:
def get_rename_dict(df, remove_numbers, rm_numbers = False):
    old = df.columns
    if rm_numbers == False:
        new = [colstring.lower() for colstring in df.columns]
    else:
        new = [remove_numbers(colstring).lower() for colstring in df.columns]
    return dict(zip(old, new))

In [7]:
rename_dict = get_rename_dict(imd_df_raw, remove_numbers, rm_numbers = True)
rename_dict

{'FID': 'fid',
 'LSOA11CD': 'lsoacd',
 'LSOA11NM': 'lsoanm',
 'LAD19CD': 'ladcd',
 'LAD19NM': 'ladnm',
 'IMD19': 'imd'}

In [8]:
imd_df = (
    imd_df_raw
    .rename(rename_dict)
    .select(pl.all().exclude('fid'))
    .filter(pl.col('lsoacd').is_in(ca_lsoa_codes))
)

imd_df.shape

(9028, 5)

In [10]:
n_unmatched_lsoas = len(ca_lsoa_codes) - imd_df.shape[0]
n_unmatched_lsoas

419